In [2]:
#Import the necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)

Using TensorFlow backend.


In [0]:
#set seed to make the result reproducible 
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

## Load the cifar dataset

In [0]:
download data
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()

#labels=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

#labels 0,1,8,9 --->  Transport
#labels 2,3,4,5,6,7 ----> Animals
#For training data combine the 10 classes to two classes,0 for transport and 1 for animals

y_train[np.where(np.isin(y_train,[0,1,8,9]))] = 0
y_train[np.where(np.isin(y_train,[2,3,4,5,6,7]))] = 1

#two classes labels
labels = ['Transport','Animal']


#For test data combine the 10 classes to two classes,0 for transport and 1 for animals
y_test[np.where(np.isin(y_test,[0,1,8,9]))] = 0
y_test[np.where(np.isin(y_test,[2,3,4,5,6,7]))] = 1


## Normalize the inputs and reshape the labels

In [5]:
y_train = tf.keras.utils.to_categorical(y_train[:40000])
x_train = (x_train[:40000]  / 255.0)


(40000, 2)

In [0]:
#use learning rate scheduler
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)
styles=[':','-.','--','-',':','-.','--','-',':','-.','--','-']

## How many pairs of convolution-subsampling should we use?


In [7]:
# BUILD the CNN
nets = 3
model = [0] *nets

for j in range(3):
    model[j] = Sequential()
    model[j].add(Conv2D(16,kernel_size=5,padding='same',activation='relu',
            input_shape=(32,32,3)))
    model[j].add(MaxPool2D())
    if j>0:
        model[j].add(Conv2D(32,kernel_size=5,padding='same',activation='relu'))
        model[j].add(MaxPool2D())
    if j>1:
        model[j].add(Conv2D(64,kernel_size=5,padding='same',activation='relu'))
        model[j].add(MaxPool2D(padding='same'))
    model[j].add(Flatten())
    model[j].add(Dense(256, activation='relu'))
    model[j].add(Dense(2, activation='softmax'))
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

W0720 00:11:43.521624 140308054640512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0720 00:11:43.524361 140308054640512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0720 00:11:43.529478 140308054640512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0720 00:11:43.553358 140308054640512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0720 00:11:43.584976 140308054640512 deprecation_wrapp

In [20]:
#Compile and train
# Divide training data to training and validation datasets
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(x_train, y_train, test_size = 0.2)
# Train the CNN
history = [0] * nets
names = ["(C-P)x1","(C-P)x2","(C-P)x3"]
epochs = 20
for j in range(nets):
    history[j] = model[j].fit(X_train2,Y_train2, batch_size=128, epochs = epochs, 
        validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
    print("CNN {0}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        names[j],epochs,max(history[j].history['acc']),max(history[j].history['val_acc']) ))

W0719 00:18:44.426304 140086052452224 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0719 00:18:44.481204 140086052452224 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



CNN (C-P)x1: Epochs=20, Train accuracy=0.99203, Validation accuracy=0.92488
CNN (C-P)x2: Epochs=20, Train accuracy=0.99675, Validation accuracy=0.93037
CNN (C-P)x3: Epochs=20, Train accuracy=0.99950, Validation accuracy=0.92888


## How many Feature maps for each layer?

In [8]:
# Build the CNN
nets = 6
model = [0] *nets
for j in range(6):
    model[j] = Sequential()
    model[j].add(Conv2D(j*8+8,kernel_size=5,activation='relu',input_shape=(32,32,3)))
    model[j].add(MaxPool2D())
    model[j].add(Conv2D(j*16+16,kernel_size=5,activation='relu'))
    model[j].add(MaxPool2D())
    model[j].add(Flatten())
    model[j].add(Dense(256, activation='relu'))
    model[j].add(Dense(2, activation='softmax'))
    model[j].compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

W0720 00:11:48.951632 140308054640512 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [23]:
# Divide train dataset to training and validation datasets
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(x_train, y_train, test_size = 0.2)
# Train the CNN
history = [0] * nets
names = ["8 maps","16 maps","24 maps","32 maps","48 maps","64 maps"]
epochs = 30
for j in range(nets):
    history[j] = model[j].fit(X_train2,Y_train2, batch_size=128, epochs = epochs, 
        validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
    print("CNN {0}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        names[j],epochs,max(history[j].history['acc']),max(history[j].history['val_acc']) ))

CNN 8 maps: Epochs=30, Train accuracy=0.99956, Validation accuracy=0.94050
CNN 16 maps: Epochs=30, Train accuracy=0.99997, Validation accuracy=0.96413
CNN 24 maps: Epochs=30, Train accuracy=1.00000, Validation accuracy=0.96800
CNN 32 maps: Epochs=30, Train accuracy=1.00000, Validation accuracy=0.96913
CNN 48 maps: Epochs=30, Train accuracy=1.00000, Validation accuracy=0.96788
CNN 64 maps: Epochs=30, Train accuracy=0.99997, Validation accuracy=0.96888


## How many units used for the dense layer?

In [0]:
# Build the CNN
nets = 8
model = [0] *nets

for j in range(8):
    model[j] = Sequential()
    model[j].add(Conv2D(32,kernel_size=5,activation='relu',input_shape=(32,32,3)))
    model[j].add(MaxPool2D())
    model[j].add(Conv2D(64,kernel_size=5,activation='relu'))
    model[j].add(MaxPool2D())
    model[j].add(Flatten())
    if j>0:
        model[j].add(Dense(2**(j+4), activation='relu'))
    model[j].add(Dense(2, activation='softmax'))
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [26]:
# Divid the train data into training and validation datasets
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(x_train, y_train, test_size = 0.2)
# Train the CNN
history = [0] * nets
names = ["0N","32N","64N","128N","256N","512N","1024N","2048N"]
epochs = 30
for j in range(nets):
    history[j] = model[j].fit(X_train2,Y_train2, batch_size=128, epochs = epochs, 
        validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
    print("CNN {0}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        names[j],epochs,max(history[j].history['acc']),max(history[j].history['val_acc']) ))

CNN 0N: Epochs=30, Train accuracy=0.98859, Validation accuracy=0.94312
CNN 32N: Epochs=30, Train accuracy=0.99994, Validation accuracy=0.95337
CNN 64N: Epochs=30, Train accuracy=0.99994, Validation accuracy=0.95562
CNN 128N: Epochs=30, Train accuracy=0.99975, Validation accuracy=0.93088
CNN 256N: Epochs=30, Train accuracy=0.99738, Validation accuracy=0.92737
CNN 512N: Epochs=30, Train accuracy=0.99997, Validation accuracy=0.92900
CNN 1024N: Epochs=30, Train accuracy=1.00000, Validation accuracy=0.93375
CNN 2048N: Epochs=30, Train accuracy=0.99997, Validation accuracy=0.92637


## How much dropout to use after each Conv-pool layer?

In [10]:
# Build the CNN
nets = 8
model = [0] *nets

for j in range(8):
    model[j] = Sequential()
    model[j].add(Conv2D(32,kernel_size=5,activation='relu',input_shape=(32,32,3)))
    model[j].add(MaxPool2D())
    model[j].add(Dropout(j*0.1))
    model[j].add(Conv2D(64,kernel_size=5,activation='relu'))
    model[j].add(MaxPool2D())
    model[j].add(Dropout(j*0.1))
    model[j].add(Flatten())
    model[j].add(Dense(64, activation='relu'))
    model[j].add(Dropout(j*0.1))
    model[j].add(Dense(2, activation='softmax'))
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

W0720 00:11:59.509644 140308054640512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0720 00:11:59.518380 140308054640512 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0720 00:12:00.287987 140308054640512 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0720 00:12:00.319332 140308054640512 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ens

In [28]:
# Divide the training data into training and validation datasets
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(x_train, y_train, test_size = 0.2)
# train the network
history = [0] * nets
names = ["D=0","D=0.1","D=0.2","D=0.3","D=0.4","D=0.5","D=0.6","D=0.7"]
epochs = 30
for j in range(nets):
    history[j] = model[j].fit(X_train2,Y_train2, batch_size=128, epochs = epochs, 
        validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
    print("CNN {0}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        names[j],epochs,max(history[j].history['acc']),max(history[j].history['val_acc']) ))

CNN D=0: Epochs=30, Train accuracy=0.99866, Validation accuracy=0.92963
CNN D=0.1: Epochs=30, Train accuracy=0.97891, Validation accuracy=0.93800
CNN D=0.2: Epochs=30, Train accuracy=0.96862, Validation accuracy=0.94050
CNN D=0.3: Epochs=30, Train accuracy=0.95778, Validation accuracy=0.93763
CNN D=0.4: Epochs=30, Train accuracy=0.94644, Validation accuracy=0.93988
CNN D=0.5: Epochs=30, Train accuracy=0.93581, Validation accuracy=0.93712
CNN D=0.6: Epochs=30, Train accuracy=0.92266, Validation accuracy=0.92875
CNN D=0.7: Epochs=30, Train accuracy=0.91087, Validation accuracy=0.91625


# Choose between different models 

In [0]:
j=4
model[j] = Sequential()

model[j].add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(32,32,3)))
model[j].add(BatchNormalization())
model[j].add(Conv2D(32,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))

model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(64,kernel_size=3,activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))

model[j].add(Flatten())
model[j].add(Dense(64, activation='relu'))
model[j].add(BatchNormalization())
model[j].add(Dropout(0.4))
model[j].add(Dense(2, activation='softmax'))

model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [14]:
# Divide data into training and validation datasets
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(x_train, y_train, test_size = 0.2)
#Traing 4 different models with diffrent features
history = [0] * nets
names = ["basic","32C3-32C3","32C5S2","both+BN","both+BN+DA"]
epochs = 50
for j in range(nets-1):
    history[j] = model[j].fit(X_train2,Y_train2, batch_size=128, epochs = epochs,  
        validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
    print("CNN {0}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        names[j],epochs,max(history[j].history['acc']),max(history[j].history['val_acc']) ))
    
# Use data augmentation to create more training data
datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.1,  
        width_shift_range=0.1, 
        height_shift_range=0.1)
# Train the last netwrok with all the features added
j = nets-1
history[j] = model[j].fit_generator(datagen.flow(X_train2,Y_train2, batch_size=128), 
    epochs = epochs, steps_per_epoch = X_train2.shape[0]//128,
    validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
print("CNN {0}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
    names[j],epochs,max(history[j].history['acc']),max(history[j].history['val_acc']) ))

CNN basic: Epochs=50, Train accuracy=0.99803, Validation accuracy=0.92650
CNN 32C3-32C3: Epochs=50, Train accuracy=0.99906, Validation accuracy=0.96788
CNN 32C5S2: Epochs=50, Train accuracy=0.99275, Validation accuracy=0.96725
CNN both+BN: Epochs=50, Train accuracy=0.98059, Validation accuracy=0.95813
CNN both+BN+DA: Epochs=50, Train accuracy=0.99591, Validation accuracy=0.95788


KeyboardInterrupt: ignored